<a href="https://colab.research.google.com/github/ayushraanjan/NLP-FineTuned/blob/main/fine_tunning_BERT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine Tunning bert-base-uncased for stanford/sst2 dataset

In [1]:
from transformers import AutoTokenizer, DataCollatorWithPadding
!pip install datasets
from datasets import load_dataset

raw_datasets = load_dataset("stanfordnlp/sst2")
raw_datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label'],
        num_rows: 1821
    })
})

### Using bertweet-base-sentiment-analysis model


In [2]:
from transformers import AutoModelForSequenceClassification

checkpoint = "finiteautomata/bertweet-base-sentiment-analysis"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [4]:
def tokenizer_function(example):
  return tokenizer(example["sentence"], truncation = True)


In [5]:
tokenized_dataset = raw_datasets.map(tokenizer_function, batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [7]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
tokenized_dataset = tokenized_dataset.remove_columns(["idx", "sentence"])
tokenized_dataset = tokenized_dataset.rename_column('label', 'labels')
tokenized_dataset.set_format('torch')


In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [34]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset['train'], shuffle=True, batch_size =12, collate_fn= data_collator,
)
val_dataloader = DataLoader(
    tokenized_dataset['validation'], shuffle=True, batch_size=1, collate_fn=data_collator,
)
test_dataloader = DataLoader(
    tokenized_dataset['test'], shuffle=True, batch_size=12, collate_fn=data_collator,
)

In [11]:
for batch in train_dataloader:
  break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([12]),
 'input_ids': torch.Size([12, 21]),
 'token_type_ids': torch.Size([12, 21]),
 'attention_mask': torch.Size([12, 21])}

In [12]:
output = model(**batch)
print(output.loss, output.logits.shape)

tensor(0.7463, grad_fn=<NllLossBackward0>) torch.Size([12, 3])


In [13]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs*len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,

)
print(num_training_steps)

16839


In [15]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)
device

device(type='cuda')

In [16]:
from tqdm.auto import tqdm

progress = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    output = model(**batch)
    loss = output.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress.update(1)





  0%|          | 0/16839 [00:00<?, ?it/s]

In [20]:
!pip install evaluate

import evaluate
import torch

# Load the accuracy metric
metric = evaluate.load("accuracy")

model.eval()

for batch in val_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        output = model(**batch)
    logits = output.logits
    predictions = torch.argmax(logits, dim=-1)

    # Add predictions and references (true labels) to the metric
    metric.add_batch(predictions=predictions, references=batch['labels'])

# Compute and print the final metric results
results = metric.compute()
print(f"Accuracy: {results['accuracy']:.4f}")


Accuracy: 0.9323


### For Test Set

In [35]:
metric = evaluate.load("accuracy")
model.eval()

for batch in test_dataloader:
  batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}

  with torch.no_grad():
    output = model(**batch)
  logits = output.logits
  predictions = torch.argmax(logits, dim=-1)
  metric.add_batch(predictions=predictions, references= batch['labels'].squeeze())

results = metric.compute()
print(f"Accuracy: {results['accuracy']:.4f}")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [37]:
metric = evaluate.load("accuracy")
model.eval()

for batch in test_dataloader:
  # Only move tensors that are expected by the model to the device
  # Assuming 'input_ids', 'attention_mask', and 'labels' are the required keys
  batch = {
      'input_ids': batch['input_ids'].to(device),
      'attention_mask': batch['attention_mask'].to(device),
      'labels': batch['labels'].to(device),
  }

  with torch.no_grad():
    output = model(**batch)
  logits = output.logits
  predictions = torch.argmax(logits, dim=-1)
  metric.add_batch(predictions=predictions, references=batch['labels'])

results = metric.compute()
print(f"Accuracy: {results['accuracy']:.4f}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [38]:
import evaluate
import torch

metric = evaluate.load("accuracy")
model.eval()

for batch in test_dataloader:
  # Only move tensors that are expected by the model to the device
  # Assuming 'input_ids', 'attention_mask', and 'labels' are the required keys
  batch = {
      'input_ids': batch['input_ids'].to(device),
      'attention_mask': batch['attention_mask'].to(device),
      'labels': batch['labels'].to(device),
  }

  # Check the shapes of the tensors before passing to the model
  print(f"Input IDs shape: {batch['input_ids'].shape}")
  print(f"Attention Mask shape: {batch['attention_mask'].shape}")
  print(f"Labels shape: {batch['labels'].shape}")

  with torch.no_grad():
    output = model(**batch)
  logits = output.logits
  predictions = torch.argmax(logits, dim=-1)
  metric.add_batch(predictions=predictions, references=batch['labels'])

results = metric.compute()
print(f"Accuracy: {results['accuracy']:.4f}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [32]:
print(device)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))


cuda
True
0
Tesla T4
